In [ ]:
%%capture
!git clone https://@github.com/marcin119a/sat-experiments.git
%cd sat-experiments


##### Kompilacja akmaxsat -- pakietu do rozwiązywanie MAX-SAT, w naszym przypadku 2-MAX-SAT

In [ ]:
%cd /content/sat-experiments/akmaxsat_1.1
!make
%cd /content/sat-experiments

In [ ]:
%%capture
!pip install -r requirements.txt

### Pobranie projektu z githuba generujacego zbiór danych D-SAT(n) 

In [ ]:
%%capture
!git clone https://github.com/satproject/neuralheuristicsforsat.git

### postawowe parametry symulatora SIMCim

In [ ]:
hyperparameters = {
    'N': 400, #ilość iteracji symulatora 
    'sigma': 0.2, #poziom szumu 
    'dt': 0.3, # współczynnik uczenia
    'zeta': 1, # oddziaływanie
    'attempt_num': 1000, # attempt
    'alpha': 0.2, # parametr metody SGD stochastyczne obniżanie gradientu wykorzystanej w algorytmie
    'complexity': 20, #n parametr ilosci zmiennych
    'seed': 1000, #losowy szum
    'observations': 1000 #ilosc formuł
}

### Wygenerowanie zbioru danych D-SAT(20)

In [ ]:
%%capture
!python /content/sat-experiments/neuralheuristicsforsat/dump_dataset_2sat.py -o 10000 -c 20 -j 10000

### Uruchomienie algorytmów akmaxsat SimCim i brute-force dla D-SAT(20)

In [ ]:
import os
import tensorflow as tf
import itertools
from neuralheuristicsforsat.helpers import chunkIt, save_to_csv
 
from maxsat import (
    akmaxsat_format_to_list, akmaxsat_run, function_from_akmaxsat, brute_force_2_max_sat, dump, transform
)
import time
from SIMCIM import SIMCim
import csv

params = hyperparameters
# ilosc zmiennych
n = 20 
# sciezka do pliku D-SAT(50)
tfrecord_location = '/content/sat-experiments/sr_20/train_10000_sr_20.tfrecord' 

record_iterator = tf.python_io.tf_record_iterator(path=tfrecord_location)
preds = []
batch_size = params['observations']


from importlib import reload

train_set = {'cnf': list(), 'sat': list()}
sim_results = list()

# dla 100 formuł uruchamiamy eksperyment porównujący SIMCim, brute-force i akmaxsat 
for string_record in itertools.islice(record_iterator, 100): 
    example = tf.train.Example()
    # wczytujemy dataset D-SAT(20) 
    example.ParseFromString(string_record) 

    # ilosc klazul w formule 2CNF
    m = len(example.features.feature["inputs"].float_list.value) // 2 

    # Formuły w postaci CNF: 
    inputs = chunkIt(example.features.feature["inputs"].float_list.value, m) 
    train_set['cnf'].append(inputs)
    targ = int(example.features.feature["sat"].float_list.value[0])
    train_set['sat'].append(targ)

    start = time.time()
    cnf = transform(inputs)
    # zapisanie do pliku dla algorytmu akmaxsat
    dump(inputs, n, m, round(start,2)) 

    # nazwa pliku wejscia dla algorytmu akmaxsat
    s = '/content/sat-experiments/out/{0}.txt'.format(round(start,2)) 

    # uruchomienie algorytmu akmaxsat dla tego pliku
    output = akmaxsat_run('/content/sat-experiments/akmaxsat_1.1/akmaxsat',s) 

      
    literals = set()
    for conj in cnf:
            for disj in conj:
                literals.add(disj[0])
    # sprawdzenie ile klauzul zostało spełnionych
    check_cnf = [bool(disj.intersection(output)) for disj in cnf] 
    MAX_akmaxsat = check_cnf.count(True)
    if targ == 0: 
       # uruchomienie algorytmu bruteforce tylko dla formuł które są UNSAT=0
      vars, max_2sat = brute_force_2_max_sat(cnf)
    else:
      max_2sat = m 
      
    # predykcja symulatora SIMCim
    sim_result = SIMCim(params, n, m, inputs)  

    end = time.time()
    
    # zapisanie wyników do pliku csv
    save_to_csv('max_sat_100.csv', [[m, m / n, m + sim_result, MAX_akmaxsat, max_2sat , targ, end - start]], n)